In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('/Users/amorav/Library/Mobile Documents/com~apple~CloudDocs/DS_projects/car_value_estimator/data/ads_all.csv')

I will use all the results from the Data Analysis phase here 
* Best model is RandomForest
 {'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [3]:
#remove unneccessary columns and NaN
data.drop(columns=['id', 'ad_id', 'link', 'num_of_pic','tags'], inplace=True)

#remove rows that have nan values
data.dropna(how='all', inplace=True)

In [4]:
# Prepare features of dataset for analysis
# transform price to numeric
data['price']=data['price'].apply(lambda x: float(str(x).replace(',', '')))

# Remove 'km' string and convert to numeric
data['km'] = data['km'].str.replace(' km', '').str.replace(',', '').astype(float)

# Feature Engineering
# Extract car age
current_year = 2024
data['car_age'] = current_year - data['year']

# Add warranty feature
data['title']=data['title'].str.lower()
data['warranty']=data['title'].apply(lambda x: 'warranty' in str(x))

#add luxury brand identifier
luxury_brands = ['mercedez-benz','lexus','bmw','audi']
data['luxury'] = data['brand'].apply(lambda x: 1 if x.lower() in luxury_brands else 0)

#Remove columns that won't be used in analysis
data.drop(columns=['title','location'], inplace=True)

In [5]:

# Detect and remove outliers
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.05)
    Q3 = df[column].quantile(0.95)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

# Get unique brands
brands = data['brand'].unique()

outliers_df=[]
for i, brand in enumerate(brands):
    brand_data = data[data['brand'] == brand]
    outliers = detect_outliers(brand_data, 'price')
    outliers_df.append(outliers)

outliers_df=pd.concat(outliers_df)
#Remove outliers and review the Prices per brand again
data_cleaned=data.drop(index=outliers_df.index)



### Prepare features and build the model


In [6]:
# Identify categorical columns
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

# Count unique values in each categorical column
unique_counts = data_cleaned[categorical_columns].nunique()
unique_counts



brand             12
model            361
steering_side      2
region_specs       8
dtype: int64

In [7]:

# Check for NaN values in data again
nan_columns = data_cleaned.columns[data_cleaned.isnull().any()].tolist()
print("Columns with NaN values:", nan_columns)

Columns with NaN values: ['year', 'km', 'steering_side', 'region_specs', 'car_age']


In [8]:
#Review and delete nan
print('Nan in km column')
print(data_cleaned[data_cleaned.km.isna()])


Nan in km column
         price    brand     model  year  km steering_side region_specs  \
1331   67900.0    Honda      City   NaN NaN           NaN          NaN   
1972   60000.0  Hyundai   Elantra   NaN NaN           NaN          NaN   
3180   41900.0      Kia    Cerato   NaN NaN           NaN          NaN   
6760  129000.0   Toyota  Fortuner   NaN NaN           NaN          NaN   

      car_age  warranty  luxury  
1331      NaN     False       0  
1972      NaN     False       0  
3180      NaN     False       0  
6760      NaN     False       0  


In [9]:
#Remove nan
data_cleaned=data_cleaned[~data_cleaned.km.isna()]

In [10]:
# Check for NaN values in data again
nan_columns = data_cleaned.columns[data_cleaned.isnull().any()].tolist()
print("Columns with NaN values:", nan_columns)

Columns with NaN values: []


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from joblib import dump
import numpy as np

# Define the preprocessing steps for numerical and categorical data
numerical_features = ['km', 'car_age']
categorical_features = ['brand', 'model', 'steering_side', 'region_specs', 'luxury']
boolean_features = ['warranty']

# Create numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define a custom transformer to convert categorical variables to lower case
class LowerCaseTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        lowercase_func = np.vectorize(lambda s: s.lower() if isinstance(s, str) else s)
        X = lowercase_func(X)
        return X

# Create categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('lowercase', LowerCaseTransformer()),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define a custom transformer for boolean features
class BoolToIntTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.astype(int)

# Create boolean pipeline
boolean_pipeline = Pipeline(steps=[
    ('bool_to_int', BoolToIntTransformer()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

# Combine both pipelines into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('bool', boolean_pipeline, boolean_features)
    ])

# Create the pipeline with the preprocessor and DecisionTreeRegressor model
#Replace 300 to 10 to speed up the process
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        bootstrap=True,
        ccp_alpha=0.0,
        criterion='squared_error',
        max_depth=None,
        max_features=1.0,
        max_leaf_nodes=None,
        max_samples=None,
        min_impurity_decrease=0.0,
        min_samples_leaf=1,
        min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        n_estimators=10,#300,
        n_jobs=None,
        oob_score=False,
        random_state=42,
        verbose=0,
        warm_start=False
    ))
])




In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

# Define features and target
X = data_cleaned.drop('price', axis=1)
y = data_cleaned['price']

# Perform k-fold cross-validation
cv_score = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_score=np.abs(cv_score)
print('CV scores')
print(cv_score)
avg_cv_score=np.mean(cv_score)
print(f'Average CV mean_absolute_error {avg_cv_score}')

CV scores
[32818.72841989 14843.05114409 13779.48476684 17240.69890075
 28733.99196517 38506.48672751 43650.66032104 55941.99985858
 60903.2109099  46491.53674629]
Average CV mean_absolute_error 35290.98497600573


### Define the records that are constantly predicted with high error and dig into problem

In [13]:
#Define the records that are constantly predicted with high error
# Calculate the number of rows corresponding to 80%
n_rows = X.shape[0]
n_80_percent = int(n_rows * 0.8)

bad_pred_indexes=set()
for i in range(10):
    # Select 80% of the indexes
    selected_indexes = np.random.choice(X.index, n_80_percent, replace=False)
    pred_indexes=list(set(X.index)-set(selected_indexes))
    
    X_train, X_test, y_train, y_test=X.loc[selected_indexes], X.loc[pred_indexes], y.loc[selected_indexes], y.loc[pred_indexes]
    
    best_model=pipeline.fit(X_train, y_train)
    y_hat=pipeline.predict(X_test)
    
    prediction=pd.DataFrame({'y_test':y_test.values,'y_hat': y_hat}, index=pred_indexes)
    prediction['mae']=np.abs(prediction['y_hat']-prediction['y_test'])
    prediction=prediction[prediction.mae>50000].sort_values('mae')
    
    bad_pred_indexes.update(set(prediction.index.values))
    

In [14]:
len(bad_pred_indexes)

2699

In [15]:
#Define the records with high pred errors
high_error_pred=data_cleaned.loc[list(bad_pred_indexes)].groupby('brand')['model'].value_counts().reset_index().rename(columns={'count':'high_error'})
all_counts=data_cleaned.groupby('brand')['model'].value_counts().reset_index()
merged_df=all_counts.merge(high_error_pred).sort_values('high_error')
merged_df['perc_high_error']=merged_df['high_error']/merged_df['count']
merged_df.sort_values('perc_high_error').tail(20)
print('Ratio of items that correspond to high error')
high_error_category=merged_df[merged_df.perc_high_error>0.5]
bad_pred_sum=high_error_category['count'].sum()
print("{:.2%}".format(bad_pred_sum/data_cleaned.shape[0]))
#data_cleaned_.loc[list(bad_pred_indexes)].groupby('brand')['model'].value_counts().sort_values(ascending=False).head(30)

Ratio of items that correspond to high error
6.25%


In [16]:
high_error_category

,brand,model,count,high_error,perc_high_error
220,Volkswagen,Eurovan,1,1,1.000000
126,Lexus,SC-Series,1,1,1.000000
165,Mercedes-Benz,240/260/280,1,1,1.000000
166,Mercedes-Benz,450 SEL,1,1,1.000000
167,Mercedes-Benz,EQA,1,1,1.000000
67,Chevrolet,Apache,1,1,1.000000
69,Chevrolet,SSR,1,1,1.000000
90,Ford,Thunderbird,1,1,1.000000
89,Ford,Other,1,1,1.000000
68,Chevrolet,Nova,1,1,1.000000


We'll remove it from analysis and test model again

In [17]:
remove_records=data_cleaned.merge(high_error_category[['brand', 'model']], how='right')
data_cleaned_ = data_cleaned[~data_cleaned.isin(remove_records.to_dict(orient='list')).all(axis=1)]


In [18]:
data_cleaned[data_cleaned.brand.str.lower()=='mercedes-benz'][['brand','model']].drop_duplicates()
np.sort(data_cleaned[data_cleaned.brand.str.lower()=='volkswagen']['model'].unique())
#data_cleaned.brand.unique()
#'Honda', 'Hyundai', 'Kia', 'Nissan', 'Toyota',  'Chevrolet', v 'Volkswagen'

array(['Arteon', 'Beetle', 'Bora', 'CC', 'Caddy', 'Crafter', 'Eos',
       'Eurovan', 'GTI', 'Golf', 'Golf R', 'ID.3', 'ID.4', 'ID.6', 'ID.7',
       'Jetta', 'Multivan', 'Other', 'Passat', 'Polo', 'Scirocco',
       'T-Roc', 'Teramont', 'Tiguan', 'Touareg', 'Transporter'],
      dtype=object)

In [19]:
data_cleaned_

,price,brand,model,year,km,steering_side,region_specs,car_age,warranty,luxury
0,159000.0,Ford,F-Series Pickup,2018.0,107000.0,Left Hand,GCC Specs,6.0,False,0
1,276000.0,Ford,Mustang,2021.0,2000.0,Left Hand,American Specs,3.0,True,0
2,62500.0,Ford,Edge,2016.0,79000.0,Left Hand,GCC Specs,8.0,False,0
3,169000.0,Ford,Mustang,2020.0,55000.0,Left Hand,GCC Specs,4.0,True,0
4,209000.0,Ford,Mustang,2023.0,29000.0,Left Hand,GCC Specs,1.0,True,0
...,...,...,...,...,...,...,...,...,...,...
21117,26000.0,Volkswagen,Passat,2014.0,107624.0,Left Hand,GCC Specs,10.0,False,0
21118,110000.0,Volkswagen,Tiguan,2023.0,0.0,Left Hand,Chinese Specs,1.0,False,0
21119,79000.0,Volkswagen,Transporter,2019.0,0.0,Left Hand,GCC Specs,5.0,False,0
21120,185000.0,Volkswagen,Touareg,2021.0,54300.0,Left Hand,GCC Specs,3.0,False,0


In [20]:
# Define features and target
X = data_cleaned_.drop('price', axis=1)
y = data_cleaned_['price']

# Perform k-fold cross-validation
cv_score = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_score=np.abs(cv_score)
print('CV scores')
print(cv_score)
avg_cv_score=np.mean(cv_score)
print(f'Average CV mean_absolute_error {avg_cv_score}')

CV scores
[33966.15153505 13535.15232893 13216.43361    14346.22685626
 16960.96095239 28265.03255717 37653.2136726  29567.29496211
 36668.465834   32846.97327431]
Average CV mean_absolute_error 25702.590558281834


### Create pipeline for the best model and store it

In [21]:
# Fit the pipeline
pipeline.fit(X, y)

# Save the model
#dump(pipeline, 'models/random_forest_model.pkl')



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['km', 'car_age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='Unknown',
                                                                                 strategy='constant')),
                                                                  ('lowercase',
                                                                   LowerCaseTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['brand', 'model',
                                                   'steering_side',
                                                   'region_specs', 'luxury']),
                                                 ('bool',
                                                  Pipeline(steps=[('bool_to_int',
                                                                   BoolToIntTransformer()),
                                                                  ('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['warranty'])])),
                ('model',
                 RandomForestRegressor(n_estimators=10, random_state=42))])

In [22]:
from joblib import dump, load

# Save the pipeline
dump(pipeline, 'models/random_forest_model.joblib')


['models/random_forest_model.joblib']

In [23]:
from joblib import load

# Load the pipeline
pipeline_loaded = load('models/random_forest_model.joblib')

# Check the type of the loaded pipeline
print(type(pipeline_loaded))  # Should output: <class 'sklearn.pipeline.Pipeline'>


<class 'sklearn.pipeline.Pipeline'>


In [142]:
X.tags.unique()

array([nan, 'FEATURED'], dtype=object)

In [144]:
data=pd.read_csv('/Users/amorav/Library/Mobile Documents/com~apple~CloudDocs/DS_projects/car_value_estimator/data/ads_all.csv')
data.head()

,id,ad_id,link,num_of_pic,price,brand,model,title,year,km,steering_side,region_specs,location,tags
0,1,2b245180a2df401d91d205226a6f5889/,https://uae.dubizzle.com/motors/used-cars/ford...,17.0,"159,000",Ford,F-Series Pickup,SVT Raptor 2018 F150 under service contract Fe...,2018.0,"107,000 km",Left Hand,GCC Specs,"Sheikh Zayed Road, Dubai",NaN
1,2,c688c6c29231491796a683f7a4126006/,https://uae.dubizzle.com/motors/used-cars/ford...,18.0,"276,000",Ford,Mustang,"Ford Mustang Shelby, original, American import...",2021.0,"2,000 km",Left Hand,American Specs,Saif Zone (Sharjah International Airport Free ...,NaN
2,3,20e7537262574ffaa63989e950542f32/,https://uae.dubizzle.com/motors/used-cars/ford...,19.0,"62,500",Ford,Edge,"Black Package â€¢ 1,295x48PM â€¢ 2016 Ford Edg...",2016.0,"79,000 km",Left Hand,GCC Specs,"Al Quoz, Dubai",NaN
3,4,d4d0bd4fb6cb40a6b2f8edfd577dbc7f/,https://uae.dubizzle.com/motors/used-cars/ford...,19.0,"169,000",Ford,Mustang,"FORD WARRANTY â€¢ 2,660x60PM â€¢ 2020 Ford Mus...",2020.0,"55,000 km",Left Hand,GCC Specs,"Al Quoz, Dubai",NaN
4,5,0e70caa1d9f446a79ea846ecc8add920/,https://uae.dubizzle.com/motors/used-cars/ford...,19.0,"209,000",Ford,Mustang,"Ford Warranty â€¢ 3,290x60PM â€¢ 2023 Ford Mus...",2023.0,"29,000 km",Left Hand,GCC Specs,"Al Quoz, Dubai",NaN
